<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/XGBoost_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
#import chart_studio
#import chart_studio.plotly as py 

In [ ]:
# 데이터 가져오기
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly_champagne_sales.csv

--2022-05-15 06:10:28--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly_champagne_sales.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1596 (1.6K) [text/plain]
Saving to: ‘monthly_champagne_sales.csv’

monthly_champagne_s 100%[===================>]   1.56K  --.-KB/s    in 0s      

2022-05-15 06:10:28 (34.8 MB/s) - ‘monthly_champagne_sales.csv’ saved [1596/1596]



In [ ]:
data = pd.read_csv('monthly_champagne_sales.csv')
dates = pd.date_range(start='1964-01-01', freq='MS', periods=len(data))

yy = []
mm = []

date_list = dates.tolist()
for w in date_list:
  yy.append(w.year)
  mm.append(w.month)

print(yy, mm)

[1964, 1964, 1964, 1964, 1964, 1964, 1964, 1964, 1964, 1964, 1964, 1964, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1965, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1966, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1967, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1968, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1970, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1971, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
data.drop('Month',axis=1,inplace=True)

data.insert(0, 'Year', yy)
data.insert(0, 'Month', mm)

print(data)



print(data)

     Month  Year  Sales
0        1  1964   2815
1        2  1964   2672
2        3  1964   2755
3        4  1964   2721
4        5  1964   2946
..     ...   ...    ...
100      5  1972   4618
101      6  1972   5312
102      7  1972   4298
103      8  1972   1413
104      9  1972   5877

[105 rows x 3 columns]
     Month  Year  Sales
0        1  1964   2815
1        2  1964   2672
2        3  1964   2755
3        4  1964   2721
4        5  1964   2946
..     ...   ...    ...
100      5  1972   4618
101      6  1972   5312
102      7  1972   4298
103      8  1972   1413
104      9  1972   5877

[105 rows x 3 columns]


In [ ]:
X, y =  data.loc[:,['Month', 'Year']].values, data.loc[:,'Sales'].values
data_dmatrix = xgb.DMatrix(X,label=y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_test, y_test)

[[   6 1968]
 [   5 1966]
 [   4 1969]
 [   4 1972]
 [  10 1971]
 [   7 1971]
 [   9 1964]
 [   6 1964]
 [   1 1964]
 [   3 1969]
 [   2 1971]
 [   5 1964]
 [   8 1966]
 [   4 1971]
 [   2 1965]
 [   3 1967]
 [   1 1970]
 [   6 1967]
 [   7 1967]
 [   1 1966]
 [  12 1965]] [4753 3937 4276 4788 6981 4633 2922 3036 2815 4510 3162 2946 1573 4676
 2475 3718 2639 4539 3663 3113 8357]


In [ ]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)

'''
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)
             '''

"\nXGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\n             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,\n             importance_type='gain', interaction_constraints='',\n             learning_rate=0.1, max_delta_step=0, max_depth=5,\n             min_child_weight=1, missing=nan, monotone_constraints='()',\n             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,\n             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,\n             tree_method='exact', validate_parameters=1, verbosity=None)\n             "

In [ ]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)


In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

RMSE: 434.191791


In [ ]:
# Creating an empty Dataframe with column names only
result = pd.DataFrame(columns=['Time', 'Test', 'Predicted'])
result['Time'] = pd.date_range(start='1/1/2000', periods=21, freq='M')
result['Test'] = y_test
result['Predicted'] = predictions
#Using Plotly to build the graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=result['Time'], y=result['Test'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=result['Time'], y=result['Predicted'],
                    mode='lines',
                    name='Predicted'))

# Edit the layout
fig.update_layout(title='Test vs Predicted Values',
                   xaxis_title='Months',
                   yaxis_title='Sales ($)',
                   template='gridon')

fig.show()